LAB 2
1. Create m x u matrix with movies as row and users as column

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Lets start by reading our dataset into a data frame

In [3]:
mov = np.genfromtxt('movies.dat',dtype=str,
                     delimiter='::',invalid_raise=False,autostrip=True)
movies = pd.DataFrame(mov,columns=['MovieID','Titles','Genres'])
movies.head()

,MovieID,Titles,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Read ratings data

In [5]:
rat = np.genfromtxt('ratings.dat',dtype=int,
                     delimiter='::',invalid_raise=False,autostrip=True)
ratings = pd.DataFrame(rat,columns=['UserID','MovieID','Ratings','Time'])
ratings.head()

,UserID,MovieID,Ratings,Time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


2. Normalize the matrix

Now we take mean for every user and calculate mean centered ratings

In [8]:
Mean = ratings.groupby(['UserID'],as_index=False,sort=False).mean().rename(columns={'Ratings':'ratings_mean'})
Mean.shape

(6040, 4)

In [ ]:
Ratings = pd.merge(ratings,Mean,on='UserID',how='left',sort='False')
Ratings['ratings_adjusted'] = Ratings['Ratings']-Ratings['ratings_mean']
Ratings.head()

In [10]:
result = Ratings[['UserID','MovieID_x','ratings_adjusted']].rename(columns={'MovieID_x':'movieID','ratings_adjusted':'ratings'})
#result = result.rename(columns={'MovieID_x':'movieID','ratings_adjusted':'ratings'})
result.head()



UserID,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,
1,0.811321,0.0,0.0,0.0,0.0,0.098592,0.0,0.115108,1.264151,0.885287,...,0.0,-0.134615,0.0,0.0,1.389286,0.000000,0.0,0.0,0.0,-0.577713
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.885287,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,-1.610714,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,-0.884892,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,-0.610714,-1.302928,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,-1.610714,0.000000,0.0,0.0,0.0,0.000000


Create the mxn matric using mean centered ratings

In [16]:
m_n_array = result.pivot_table(index='movieID',columns='UserID',values='ratings').fillna(0)
m_n_array.head()

UserID,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,
1,0.811321,0.0,0.0,0.0,0.0,0.098592,0.0,0.115108,1.264151,0.885287,...,0.0,-0.134615,0.0,0.0,1.389286,0.000000,0.0,0.0,0.0,-0.577713
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.885287,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,-1.610714,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,-0.884892,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,-0.610714,-1.302928,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,-1.610714,0.000000,0.0,0.0,0.0,0.000000


3) Compute SVD to get U, S and V

In [13]:
U,S,V = np.linalg.svd(m_n_array,full_matrices=False)
print(U.shape,S.shape,V.shape)

(3706, 3706) (3706,) (3706, 6040)


Check if reconstructed matrix is close to original

In [18]:
X_r = np.dot(U * S, V)
np.allclose(np.array(m_n_array),X_r)

True

Reduce components to 50

In [19]:
U_ = U[:,:50]
S_ = S[:50]
V_ = V[:50,:]
print(U_.shape,S_.shape,V_.shape)

(3706, 50) (50,) (50, 6040)


Reconstruct original matrix using these components

In [20]:
X_a = np.dot(U_ * S_, V_)

Now use this matrix to create a new Ratings data frame which would be filled now with reconstructed ratings

In [21]:
Ratings_df = pd.DataFrame(X_a,index=m_n_array.index,columns=m_n_array.columns)

Ratings_df.head()

UserID,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,
1,0.200819,0.158546,-0.045036,0.107061,0.126907,0.033990,0.022085,-0.161090,0.460981,0.849373,...,0.054286,-0.039768,-0.079748,0.006442,0.505824,0.352771,-0.009452,0.137603,0.109062,0.150747
2,0.019668,-0.067081,-0.121081,-0.021571,-0.009639,-0.008743,0.011145,0.022664,-0.035976,0.182582,...,-0.013148,0.039450,0.078257,-0.016548,-0.206989,-0.329961,0.013083,0.040019,-0.127305,-0.061000
3,0.010515,-0.010991,0.040105,0.044495,0.157762,0.002348,0.001125,-0.126046,0.004052,0.052477,...,-0.005350,-0.044529,-0.002421,-0.035992,-0.217575,0.259123,0.025514,-0.089279,-0.056868,-0.143872
4,-0.017453,-0.007955,0.032430,0.025665,-0.023892,-0.021449,-0.013636,-0.037242,0.004218,0.055186,...,-0.038094,-0.028616,0.028232,0.003937,-0.153174,-0.009796,-0.078630,0.010320,0.023202,0.027306
5,0.000377,0.012551,0.009847,0.036128,0.045379,0.026352,-0.006888,-0.029095,0.025659,0.053353,...,-0.005688,-0.066364,0.028180,-0.005455,-0.258519,0.032265,-0.005613,0.002095,-0.018277,0.080315


5) Implement a function that take movieID as input and then implement
cosine similarity along with sorting to recommend top 10 movies.

In [23]:
def Print_10_Similar_movies(m_id,R_data):
    allmovies = R_data.index
    movie_x = R_data.ix[m_id]
    denominator1 = np.sqrt(sum([np.square(x) for x in movie_x ]))

    cos_sim = [(m_id,1)]
    for m in allmovies:
        if m==m_id:
            continue
        else:
            mov = R_data.ix[m]
            numerator = [x*y for x,y in zip(movie_x,mov)]
            denominator2= np.sqrt(sum([np.square(i) for i in mov]))
            costheta=sum(numerator)/(denominator1*denominator2)
            cos_sim.append((m,costheta))



    cos_sim.sort(key=lambda x: x[1],reverse=True)
    similarusers=[cos_sim[i][0] for i in range(11)]
    print('Top 10 similar movies for:',movies.loc[movies['MovieID'] == str(m_id),'Titles'].item())
    print('MovieID   Titles')
    for i in similarusers:
        if i ==m_id: continue
        print(movies.loc[movies['MovieID'] == str(i),'MovieID'].item(),movies.loc[movies['MovieID'] == str(i),'Titles'].item())                       
    pass                 
                                        



Calculate top 10 ratings for Batman Forever

In [24]:
Print_10_Similar_movies(153,Ratings_df)

Top 10 similar movies for: Batman Forever (1995)
MovieID   Titles
1562 Batman & Robin (1997)
2616 Dick Tracy (1990)
1377 Batman Returns (1992)
208 Waterworld (1995)
839 Crow: City of Angels, The (1996)
168 First Knight (1995)
552 Three Musketeers, The (1993)
2153 Avengers, The (1998)
3274 Single White Female (1992)
485 Last Action Hero (1993)


Lets try with PCA NOW

6) We calculate eigen values and eigen vectors from covariance matrix

In [25]:
#calculating covariance matrix
cov_mat = np.cov(np.array(m_n_array).T)

#calculating eigen values,eigen vectors from covariance matrix
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

eig_vecs.shape

(6040, 6040)

In [26]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in range(10):
    print(eig_pairs[i][0])

Eigenvalues in descending order:
18.403734597167123
4.592420078963693
2.7173696153297633
2.525069584685649
2.206244951113136
1.978519465534591
1.6152956666330374
1.4666069336634184
1.3484703752893916
1.3108452211981836


Now we just want to take the top 50 eigen vectors according to the eigen values which would give us the maximum variance in the data. So we compute eigen vector matrix
6040x50 to map to our new space of 3070 movies x 50 components

In [30]:
res = eig_pairs[0][1].reshape(6040,1)
for i in range(1,50):
    new_col = eig_pairs[i][1].reshape(6040,1)
    res = np.append(res, new_col, axis=1)
#res is our eigen vector matrix now

array([0.00342003+0.j, 0.00402179+0.j, 0.00278592+0.j, 0.00202569+0.j])

Now we map the eigen vector matrix to our original space

In [32]:
Y = np.array(m_n_array).dot(res)
Y.shape

(3706, 50)

Now we have our new space matrix with 3070 movies and 50 top components and their values. Lets get that into a data frame


In [55]:
Ratings_pca = pd.DataFrame(Y,index=m_n_array.index)
Ratings_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
movieID,,,,,,,,,,,,,,,,,,,,,
1,(19.075842863358897+0j),(-7.229918714075966+0j),(4.591844260644287+0j),(2.9114638635515226+0j),(5.371510463579753+0j),(8.901463625497115+0j),(-1.4398771242257171+0j),(-6.827338290699725+0j),(2.2673958756887114+0j),(-0.16908453686329336+0j),...,(-4.05249570442754+0j),(-0.3270729113430103+0j),(-0.14749145033917596+0j),(-0.4992014050960195+0j),(-2.750081648227313+0j),(-2.5120017615580488+0j),(-0.5586019256714688+0j),(-0.45571358819455365+0j),(-2.5361576054722805+0j),(0.11140569239800502+0j)
2,(-5.158479779209459+0j),(-2.2831077829067254+0j),(0.903934214476539+0j),(-1.051871747310248+0j),(1.53512583840437+0j),(0.2333745059166472+0j),(0.18522581195051027+0j),(0.3360952723252108+0j),(0.9816727998847473+0j),(-0.022211574688663353+0j),...,(0.31259718367241474+0j),(0.3220101721016626+0j),(-0.524070760474887+0j),(1.8981627143221373+0j),(0.7286286070829665+0j),(1.1911632215091885+0j),(-1.511615057804996+0j),(1.9849675247674754+0j),(-1.7436946283452825+0j),(0.14917861349048178+0j)
3,(-4.628217723145519+0j),(-0.8802427737855005+0j),(-0.5246226280103841+0j),(-0.04510207796736566+0j),(-0.7164095223274712+0j),(0.0768981518155732+0j),(1.0777829346193826+0j),(-0.5430873999755843+0j),(-0.27171191014772555+0j),(-1.3316921221075613+0j),...,(0.8314144619024907+0j),(-2.668280346154898+0j),(1.009689808228254+0j),(-1.4375884955631695+0j),(0.6318390296695708+0j),(-0.17343639554294796+0j),(-0.5682857473412934+0j),(-0.310971519733024+0j),(0.32253189964927986+0j),(0.5907422240895698+0j)
4,(-2.5915317603927006+0j),(0.031471800652738566+0j),(0.41094566544903144+0j),(0.03339645343145486+0j),(-1.1164903909251027+0j),(0.9973751190426648+0j),(0.6604383578188255+0j),(0.3742523916708522+0j),(-0.7858704258699194+0j),(-0.8421788254605616+0j),...,(-0.40208455644629987+0j),(0.06830957519965351+0j),(0.04653166741546408+0j),(0.1621954692973584+0j),(0.02579706032954046+0j),(-1.417771366634467+0j),(0.8476399798888129+0j),(-0.2198644984224912+0j),(0.6173528176967479+0j),(0.32620754330399515+0j)
5,(-3.604175890484317+0j),(-0.3777094536578727+0j),(0.16070091048594437+0j),(1.1114342695012798+0j),(-1.0698429453548226+0j),(0.401296134096772+0j),(1.9356616567269547+0j),(-0.28898088775931025+0j),(-0.4642637971260395+0j),(-1.1107696932269913+0j),...,(0.2456986855385414+0j),(-0.026629305100586736+0j),(0.9385866460812853+0j),(-0.2395205074607895+0j),(0.44693956341903796+0j),(-0.03790398398265934+0j),(-0.1348008949069969+0j),(0.4706741088797236+0j),(-0.25371729270033283+0j),(-0.2884229780141387+0j)


Now we use this matrix to recommend top 10 movies for Batman Forever


In [57]:
Print_10_Similar_movies(153,Ratings_pca)

Top 10 similar movies for: Batman Forever (1995)
MovieID   Titles
1562 Batman & Robin (1997)
2616 Dick Tracy (1990)
1377 Batman Returns (1992)
208 Waterworld (1995)
839 Crow: City of Angels, The (1996)
168 First Knight (1995)
552 Three Musketeers, The (1993)
2153 Avengers, The (1998)
3274 Single White Female (1992)
485 Last Action Hero (1993)


7) Compare the results for SVD and PCA.

We see that the recommendations are same for SVD and PCA analysis for 50 components in both the cases.